In [77]:
import csv

# with open("GDA DATA.csv", "rb") as csvfile:
#     reader = csv.DictReader(csvfile)
#     for row in reader:
#         print row['Trip ID']
                        
            
            
import sqlite3

# Open the connection (this creates the database if there is not one already)
conn = sqlite3.connect('bike.db')
# Create cursor
c = conn.cursor()

# Drop all the tables if they exist
c.execute('''drop table if exists bike_share''')

# Create each table
c.execute(
    '''
    CREATE TABLE IF NOT EXISTS bike_share(
    trip_id integer,
    duration integer,
    start_date date,
    start_station text,
    start_terminal integer,
    end_date date,
    end_station text,
    end_terminal integer,
    bike_num integer,
    subscriber_type text,
    zip_code integer)
    '''    

)
# Load the data for the table from csv file
# The CSV file is opened and then turned into a python list, which is then loaded by line
with open("C:\\Users\\cjbro\\OneDrive\\Documents\\The next step\\Uber\\GDA DATA_2.csv", 'rb') as bike_data:
    reader = csv.reader(bike_data)
    next(reader, None)
    part = list(reader)
    for line in part:
        c.execute('''insert into bike_share values(?,?,?,?,?,?,?,?,?,?,?)''', line)

# Select and print one row to make sure data loaded
c.execute("select * from bike_share where trip_id = 432946")
result = c.fetchall()

for item in result:
    for thing in item:
        print (thing)


# Commit database changes and close the connection        
conn.commit()
conn.close()

432946
406
2014-08-31 22:31
Mountain View Caltrain Station
28
2014-08-31 22:38
Castro Street and El Camino Real
32
17
Subscriber
94040


In [153]:
# Open the connection (this creates the database if there is not one already)
conn = sqlite3.connect('bike.db')
# Create cursor
c = conn.cursor()

# This function gives a list of bikes which have been ridden more than 100 hours
# and shows number of hours and number of trips
def bike_hours_over_100(date, hours):
    dated = [str(date),str(hours),]
    
    print("Enter date above in the following format yyyy-mm-dd","\n")
    print("Display shows bikes with hours greater than inputed from date given","\n")

    
    c.execute('''select bike_num,  
                round((total(duration)/3600),1) as duration_hours,
                count(trip_id)
                from bike_share
                where date(start_date) > date('''+dated[0]+''') 
                group by bike_num
                having (total(duration)/3600) >= '''+dated[1]+'''
                order by duration_hours desc
                ''')
    result = c.fetchall()
    print ("Bike Num","Total Hours","\t","Number of Rides")
    for item in result:
        print (item[0],"\t",item[1],"\t"*2,item[2])


interact(bike_hours_over_100,date='2014-03-01',hours='200');

In [110]:
from __future__ import print_function
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
import sqlite3

# Open the connection (this creates the database if there is not one already)
conn = sqlite3.connect('bike.db')
# Create cursor
c = conn.cursor()


# This function take the bike number as an argument and then performs a number of
# SQL queries to collect statistics on the particular bike

def bike_facts(bike_number):
    print("Enter bike number above to recieve statistics of bike usage","\n")
    print("Bike numbers range from 9 to 878","\n")
    bikenumber = (str(bike_number),)
    # This query selects the bike number, total hours the bike has been ridden,
    # the average minutes per ride, and the total number of rides
    c.execute("""select bike_num, 
            round((total(duration)/3600),1) as duration_hours,
            round((avg(duration)/60),1),
            count(trip_id)
            from bike_share
            where bike_num = ?
            group by bike_num 
            order by duration_hours desc""", bikenumber)
    result = c.fetchall()
    print("*Bike Stats*","\n")
    for item in result:
        print ("Bike Number:"+str(item[0]))
        print ("Total Hours:"+str(item[1]))
        print ("Average Trip Minutes:"+str(item[2]))
        print ("Number of Trips:"+str(item[3]))
        print ("\n")
    
    # This query pulls the total number of trips the bike has been on each month
    print("Hours, Number of trips per Month:","\n")    
    c.execute('''select 
                    strftime("%m-%Y", start_date) as month_year,
                    round((total(duration)/3600),1) as duration_hours,
                    count(trip_id)
                    from bike_share
                    where bike_num = ?
                    group by strftime("%m-%Y", start_date)''',bikenumber)
    result2 = c.fetchall()
    for item in result2:
        print (item[0],"\t",item[1],"\t",item[2])
    print ("\n")
              
    # This query prints a list of all the origination stations of the bike
    # and how many trips originated there
    c.execute('''select distinct(start_station),count(trip_id) as stat_count 
    from bike_share where bike_num = ? group by start_station
    order by stat_count desc''', bikenumber)
    result1 = c.fetchall()

    print("Count of trips by where trip originated","\n")
    for item in result1:
        print (item[1],"\t",item[0])

    # This query prints a list of all the end stations of the bike
    # and how many trips ended there
    c.execute('''select distinct(end_station),count(trip_id) as stat_count 
    from bike_share where bike_num = ? group by end_station
    order by stat_count desc''', bikenumber)
    result1 = c.fetchall()
    print("\n")
    print("Count of trips by where trip ended","\n")
    for item in result1:
        print (item[1],"\t",item[0])

interact(bike_facts, bike_number='9');

